In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import random
import numpy as np
import pandas as pd

In [16]:
import data.data_retriever as data_retriever

In [17]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    
set_seed(0)

In [18]:
df = data_retriever.get_data('ETH')
df['unix_time'] = pd.to_datetime(df['date-time']).astype(int) / 10**9

cols_to_keep = [
    'unix_time',
    'liquidityRate_avg',
    'variableBorrowRate_avg',
    'utilizationRate_avg',
    'close_price',
    'volume',
]

crypto_df = df[cols_to_keep]
crypto_df.head()

,unix_time,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,close_price,volume
0,1.678752e+09,0.027728,0.043863,0.743329,1676.49,8843.165993
1,1.678756e+09,0.020744,0.032873,0.741720,1684.56,6320.830728
2,1.678759e+09,0.031029,0.049285,0.740579,1676.25,10817.093686
3,1.678763e+09,0.027092,0.043039,0.740210,1686.02,9007.115071
4,1.678766e+09,0.028781,0.045741,0.739932,1685.06,13505.217976


# Dataloader

In [19]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data: pd.DataFrame, window_size: int, target_columns, forecast_size: int = 1):
        self.window_size = window_size
        self.forecast_size = forecast_size
        self.target_columns = target_columns  
        
        data_array = data.values
    
        target_indices = [data.columns.get_loc(col) for col in target_columns]  

        self.X, self.y = [], []
        for i in range(len(data_array) - window_size - forecast_size + 1):
            self.X.append(data_array[i : i + window_size])
            
            if len(target_indices) == 1:
                self.y.append(data_array[i + window_size : i + window_size + forecast_size, target_indices[0]].reshape(-1, 1))
            else:
                self.y.append(data_array[i + window_size : i + window_size + forecast_size, target_indices])

        self.X = np.array(self.X)
        self.y = np.array(self.y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (torch.tensor(self.X[idx], dtype=torch.float32), 
                torch.tensor(self.y[idx], dtype=torch.float32))

In [20]:
def create_dataloaders(
        data, 
        target_columns,
        window_size=168,
        forecast_size=1,
        batch_size=32, 
        val_size=0.1,
        test_size=0.1):
    total_size = len(data)
    train_size = int((1 - val_size - test_size) * total_size)
    val_size = int(val_size * total_size)
    
    train_data = data[:train_size]
    val_data = data[train_size:train_size + val_size]
    test_data = data[train_size + val_size:]
    
    train_dataset = TimeSeriesDataset(train_data, window_size, target_columns, forecast_size)
    val_dataset = TimeSeriesDataset(val_data, window_size, target_columns, forecast_size)
    test_dataset = TimeSeriesDataset(test_data, window_size, target_columns, forecast_size)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

# Model

In [21]:
class LSTMForecaster(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size=2, forecast_size=1, dropout=0.2):
        super(LSTMForecaster, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.forecast_size = forecast_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :] 
        
        forecast = []
        for _ in range(self.forecast_size):
            out = self.fc(out)
            forecast.append(out.unsqueeze(1))
        
        forecast = torch.cat(forecast, dim=1) 
        return forecast

# Trainer

In [22]:
def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.001, device="cuda", patience=10, lr_patience=5, lr_decay_factor=0.1):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=lr_decay_factor, patience=lr_patience, verbose=True)

    best_val_loss = float("inf")
    epochs_without_improvement_es = 0
    epochs_without_improvement_lr = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)

                y_pred = model(X_val)
                loss = criterion(y_pred, y_val)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.8f}, Val Loss: {val_loss:.8f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement_es = 0
            epochs_without_improvement_lr = 0
        else:
            epochs_without_improvement_es += 1
            epochs_without_improvement_lr += 1

        if epochs_without_improvement_es >= patience:
            print(f"Early stopping at epoch {epoch+1} due to no improvement in validation loss for {patience} epochs.")
            break

        if epochs_without_improvement_lr >= lr_patience:
            print(f"Validation loss plateaued for {lr_patience} epochs. Reducing learning rate by factor of {lr_decay_factor}.")
            scheduler.step(val_loss)
            epochs_without_improvement_lr = 0

# Evaluator

In [23]:
def evaluate_model(model, test_loader, device="cuda"):
    model.to(device)
    model.eval()

    criterion = nn.MSELoss(reduction='none')
    total_samples = 0
    total_mape = torch.zeros(model.output_size, device=device) 
    total_mse = torch.zeros(model.output_size, device=device) 

    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)

            y_pred = model(X_test)

            mse_per_element = criterion(y_pred, y_test)
            mape_per_output = torch.mean(torch.abs((y_test - y_pred) / (y_test + 1e-8)), dim=(0, 1)) * 100

            total_mape += mape_per_output * X_test.size(0)
            total_samples += X_test.size(0)

            mse_per_output = mse_per_element.mean(dim=(0, 1)) 
            total_mse += mse_per_output * X_test.size(0)

    average_mse = total_mse.cpu().numpy() / total_samples
    average_mape = total_mape.cpu().numpy() / total_samples

    print(f"Test MSE (for each output dimension): {average_mse}")
    print(f"Test MAPE (for each output dimension): {average_mape}")

    return average_mse, average_mape


# Training

In [24]:
def train_and_evaluate(
        crypto_df,
        input_size=6, 
        hidden_size=32, 
        num_layers=4, 
        output_size=2, 
        forecast_size=1, 
        dropout=0.2,
        target_columns=['liquidityRate_avg', 'variableBorrowRate_avg'],
        window_size=168,
        batch_size=32,
        val_size=0.1,
        test_size=0.1,
        num_epochs=100,
        learning_rate=0.001,
        device="cuda" if torch.cuda.is_available() else "cpu",
        patience=10,
        lr_patience=5,
        lr_decay_factor=0.1,
):

    train_loader, val_loader, test_loader = create_dataloaders(
        crypto_df,
        target_columns=target_columns,
        window_size=window_size,
        forecast_size=forecast_size,
        batch_size=batch_size,
        val_size=val_size,
        test_size=test_size
    )
    
    model = LSTMForecaster(
        input_size, 
        hidden_size, 
        num_layers, 
        output_size, 
        forecast_size, 
        dropout
    ).to(device)
    
    train_model(
        model, 
        train_loader, 
        val_loader, 
        num_epochs=num_epochs, 
        learning_rate=learning_rate, 
        device=device, 
        patience=patience, 
        lr_patience=lr_patience, 
        lr_decay_factor=lr_decay_factor
    )

    evaluate_model(model, test_loader, device=device)

In [26]:
# window size of 1 day
train_and_evaluate(crypto_df, window_size=24)

C:\Users\siam\anaconda3\envs\aave\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.00045799, Val Loss: 0.00005818
Epoch 2/100, Train Loss: 0.00034283, Val Loss: 0.00004138
Epoch 3/100, Train Loss: 0.00034078, Val Loss: 0.00003982
Epoch 4/100, Train Loss: 0.00034113, Val Loss: 0.00004356
Epoch 5/100, Train Loss: 0.00033988, Val Loss: 0.00003934
Epoch 6/100, Train Loss: 0.00033923, Val Loss: 0.00004698
Epoch 7/100, Train Loss: 0.00033934, Val Loss: 0.00004278
Epoch 8/100, Train Loss: 0.00033851, Val Loss: 0.00003990
Epoch 9/100, Train Loss: 0.00033890, Val Loss: 0.00005589
Epoch 10/100, Train Loss: 0.00033863, Val Loss: 0.00006324
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 11/100, Train Loss: 0.00033897, Val Loss: 0.00004047
Epoch 12/100, Train Loss: 0.00033833, Val Loss: 0.00004401
Epoch 13/100, Train Loss: 0.00033797, Val Loss: 0.00005095
Epoch 14/100, Train Loss: 0.00033800, Val Loss: 0.00004520
Epoch 15/100, Train Loss: 0.00033813, Val Loss: 0.00003779
Epoch 16/100, Train Loss: 0.00033778, Val L

In [27]:
# window size of 1 week
train_and_evaluate(crypto_df, window_size=7*24)

C:\Users\siam\anaconda3\envs\aave\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.00037742, Val Loss: 0.00004782
Epoch 2/100, Train Loss: 0.00034429, Val Loss: 0.00005757
Epoch 3/100, Train Loss: 0.00034213, Val Loss: 0.00004439
Epoch 4/100, Train Loss: 0.00034258, Val Loss: 0.00004446
Epoch 5/100, Train Loss: 0.00034127, Val Loss: 0.00004324
Epoch 6/100, Train Loss: 0.00034071, Val Loss: 0.00004261
Epoch 7/100, Train Loss: 0.00034089, Val Loss: 0.00004267
Epoch 8/100, Train Loss: 0.00034108, Val Loss: 0.00003960
Epoch 9/100, Train Loss: 0.00034084, Val Loss: 0.00004578
Epoch 10/100, Train Loss: 0.00034026, Val Loss: 0.00004300
Epoch 11/100, Train Loss: 0.00033959, Val Loss: 0.00004292
Epoch 12/100, Train Loss: 0.00034080, Val Loss: 0.00004086
Epoch 13/100, Train Loss: 0.00034091, Val Loss: 0.00004366
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 14/100, Train Loss: 0.00033950, Val Loss: 0.00004662
Epoch 15/100, Train Loss: 0.00033951, Val Loss: 0.00004370
Epoch 16/100, Train Loss: 0.00033929, Val L

In [28]:
# window size of 1 month
train_and_evaluate(crypto_df, window_size=30*24)

C:\Users\siam\anaconda3\envs\aave\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.00050464, Val Loss: 0.00005839
Epoch 2/100, Train Loss: 0.00035216, Val Loss: 0.00005757
Epoch 3/100, Train Loss: 0.00035008, Val Loss: 0.00005679
Epoch 4/100, Train Loss: 0.00034762, Val Loss: 0.00005045
Epoch 5/100, Train Loss: 0.00034969, Val Loss: 0.00004911
Epoch 6/100, Train Loss: 0.00034705, Val Loss: 0.00004802
Epoch 7/100, Train Loss: 0.00034642, Val Loss: 0.00004838
Epoch 8/100, Train Loss: 0.00034740, Val Loss: 0.00007555
Epoch 9/100, Train Loss: 0.00034672, Val Loss: 0.00004905
Epoch 10/100, Train Loss: 0.00034743, Val Loss: 0.00005016
Epoch 11/100, Train Loss: 0.00034617, Val Loss: 0.00005101
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 12/100, Train Loss: 0.00034712, Val Loss: 0.00005048
Epoch 13/100, Train Loss: 0.00034606, Val Loss: 0.00004625
Epoch 14/100, Train Loss: 0.00034427, Val Loss: 0.00009446
Epoch 15/100, Train Loss: 0.00034908, Val Loss: 0.00004798
Epoch 16/100, Train Loss: 0.00034689, Val L